## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nova Friburgo,BR,-22.2819,-42.5311,74.19,69,30,0.94,scattered clouds
1,1,Saint George,US,37.1041,-113.5841,59.79,30,0,3.44,clear sky
2,2,Yellowknife,CA,62.4560,-114.3525,19.92,68,0,12.66,clear sky
3,3,Castro,BR,-24.7911,-50.0119,75.45,61,39,2.26,scattered clouds
4,4,Kapaa,US,22.0752,-159.3190,75.18,83,88,5.99,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Enter minimum temperature:'))
max_temp = float(input('Enter maximum temperature:'))

Enter minimum temperature:50
Enter maximum temperature:80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
filtered_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nova Friburgo,BR,-22.2819,-42.5311,74.19,69,30,0.94,scattered clouds
1,1,Saint George,US,37.1041,-113.5841,59.79,30,0,3.44,clear sky
3,3,Castro,BR,-24.7911,-50.0119,75.45,61,39,2.26,scattered clouds
4,4,Kapaa,US,22.0752,-159.3190,75.18,83,88,5.99,overcast clouds
6,6,Port Alfred,ZA,-33.5906,26.8910,61.18,83,98,5.01,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
print(len(filtered_df))
filtered_df.count()

366


City_ID                366
City                   366
Country                360
Lat                    366
Lng                    366
Max Temp               366
Humidity               366
Cloudiness             366
Wind Speed             366
Current Description    366
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_df = filtered_df.loc[~filtered_df['Country'].isna()]
filtered_df.count()

City_ID                360
City                   360
Country                360
Lat                    360
Lng                    360
Max Temp               360
Humidity               360
Cloudiness             360
Wind Speed             360
Current Description    360
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nova Friburgo,BR,74.19,scattered clouds,-22.2819,-42.5311,
1,Saint George,US,59.79,clear sky,37.1041,-113.5841,
3,Castro,BR,75.45,scattered clouds,-24.7911,-50.0119,
4,Kapaa,US,75.18,overcast clouds,22.0752,-159.3190,
6,Port Alfred,ZA,61.18,overcast clouds,-33.5906,26.8910,
7,Ribeira Grande,PT,63.00,scattered clouds,38.5167,-28.7000,
8,Busselton,AU,63.82,overcast clouds,-33.6500,115.3333,
10,Roseto Degli Abruzzi,IT,71.67,scattered clouds,42.6664,14.0227,
11,Mar Del Plata,AR,67.87,clear sky,-38.0023,-57.5575,
12,Rikitea,PF,74.25,moderate rain,-23.1203,-134.9692,


In [40]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    latitude, longitude = row['Lat'], row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    nearby_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(nearby_url, {
        'radius': params['radius'],
        'type': params['type'],
        'key': params['key'],
        'location': f'{latitude},{longitude}',
    }).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    if not len(response['results']):
        print('No information found. Skipping...')
    else:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']

No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...
No information found. Skipping...


In [41]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']

In [42]:
# 8a. Create the output File (CSV)
output_data_file = 'Vacation_Search/WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))